In [1]:
import geopandas as gpd
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gdf_osyu_100m_mesh = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_osyu_100m_mesh.shp').filter(['Meshcode','geometry'])

In [3]:
gdf_road = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/全国道路施設/drive-download-20230619T135611Z-001/道路舗装劣化データ_岩手県_utf8.csv', encoding='utf-8')

In [4]:
gdf_road

,facility_id,road_manager_c,road_manager_c_name,structures_kind,facility_name,route_name,road_manager_div,road_manager_div_name,road_manager_name,road_manager_office,...,last_repair_year,length,pavement_type,inspec_year,inspec_soundness,inspec_soundness_name,sochi_state,sochi_state_name,facility_kind,geometry
0,140276,823500,岩手河川国道事務所,舗装,0004000_435.2-435.3_1,国道4号,1,国,東北地方整備局,岩手河川国道事務所,...,2003,100.0,アスファルト,2018,3,Ⅲ,1,措置未着手,,None
1,140277,823500,岩手河川国道事務所,舗装,0004000_435.3-435.4_1,国道4号,1,国,東北地方整備局,岩手河川国道事務所,...,2003,100.0,アスファルト,2018,3,Ⅲ,1,措置未着手,,None
2,140278,823500,岩手河川国道事務所,舗装,0004000_435.4-435.5_1,国道4号,1,国,東北地方整備局,岩手河川国道事務所,...,2003,100.0,アスファルト,2018,3,Ⅲ,1,措置未着手,,None
3,140279,823500,岩手河川国道事務所,舗装,0004000_435.5-435.6_1,国道4号,1,国,東北地方整備局,岩手河川国道事務所,...,2003,100.0,アスファルト,2018,3,Ⅲ,1,措置未着手,,None
4,140280,823500,岩手河川国道事務所,舗装,0004000_435.6-435.7_1,国道4号,1,国,東北地方整備局,岩手河川国道事務所,...,2003,100.0,アスファルト,2018,3,Ⅲ,1,措置未着手,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,159331,823700,三陸国道事務所,舗装,0045031_234.9-235.0_1,国道45号（三陸沿岸道路（矢本石巻道路～野田久慈道路））,1,国,東北地方整備局,三陸国道事務所,...,2002,100.0,アスファルト,2020,3,Ⅲ,1,措置未着手,,None
2996,159332,823700,三陸国道事務所,舗装,0045031_235.0-235.1_1,国道45号（三陸沿岸道路（矢本石巻道路～野田久慈道路））,1,国,東北地方整備局,三陸国道事務所,...,2002,100.0,アスファルト,2020,3,Ⅲ,1,措置未着手,,None
2997,159333,823700,三陸国道事務所,舗装,0045031_235.1-235.2_1,国道45号（三陸沿岸道路（矢本石巻道路～野田久慈道路））,1,国,東北地方整備局,三陸国道事務所,...,2002,100.0,アスファルト,2020,3,Ⅲ,1,措置未着手,,None
2998,159334,823700,三陸国道事務所,舗装,0045031_235.2-235.3_1,国道45号（三陸沿岸道路（矢本石巻道路～野田久慈道路））,1,国,東北地方整備局,三陸国道事務所,...,2002,100.0,アスファルト,2020,3,Ⅲ,1,措置未着手,,None


In [ ]:
#道路点検データのカラム名を変更する。renameメソッドはrename()のかっこの中でcolumnsと指定しないとカラムは変更されない。
gdf_road = gdf_road.rename(columns = {'inspec_soundness':'paved_rank', 'inspec_soundness_name':'paved_rank_name'})

In [ ]:
#道路劣化調査の奥州市のみのデータフレームを作る
gdf_road = gdf_road[gdf_road['locate_area2'] == '奥州市']
gdf_road = gpd.GeoDataFrame(gdf_road)

#データフレームの緯度経度座標をstr型からflaot型に変換
gdf_road['locate_self_long'] = gdf_road['locate_self_long'].replace({'':0}).astype(float)
gdf_road['locate_self_lat'] = gdf_road['locate_self_lat'].replace({'':0}).astype(float)

#データフレームの緯度経度座標が空の行を削除する
gdf_road = gdf_road[gdf_road['locate_self_lat'] != 0]

#緯度経度座標をpoint型にしたカラムを増やす
gdf_road['geometry'] = gdf_road.apply(lambda x: Point(x['locate_self_long'], x['locate_self_lat']), axis=1)

#座標系を設定
if gdf_road.crs is None:
    # 座標系をEPSG:4326に設定
    gdf_road.set_crs(epsg=4326, inplace=True)

In [ ]:
#ポリゴンデータにポイントデータがあれば空間結合
gdf_road = gpd.sjoin(gdf_osyu_100m_mesh, gdf_road, op='contains',  how='left')
#2つのデータフレームで同じカラム（geometry)がないと結合することが出来ない

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
#indexが重複しているものを削除する
gdf_road = gdf_road[~gdf_road.index.duplicated(keep='first')]

#indexが重複しているものがないか確認
gdf_road[gdf_road.index.duplicated(keep=False)]

,Meshcode,geometry,index_right,facility_id,road_manager_c,road_manager_c_name,structures_kind,facility_name,route_name,road_manager_div,...,locate_self_long,last_repair_year,length,pavement_type,inspec_year,paved_rank,paved_rank_name,sochi_state,sochi_state_name,facility_kind


In [ ]:
#結合した際にfiltered_gdf_road_osyuのindexを削除　結合したときのindexを残したまま別のデータフレームは結合できない
gdf_road.drop(columns=['geometry', 'index_right'], axis=1, inplace=True)

In [ ]:
#出力するためのコード

# エンコードできない文字を別の形式に変換してからShift_JISでエンコードする
#gdf_5841を変えるのはめんどくさいからgdf_roadに変数を格納する
# gdf_road['paved_rank_name'] = gdf_road['paved_rank_name'].replace({'Ⅰ':'1', 'Ⅱ':2, 'Ⅲ':3 }, regex=True)  # 文字の置換
# gdf_road['column_name'] = gdf_road['column_name'].str.encode('shift_jis', errors='ignore')  # Shift_JISでエンコード


gdf_road.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市　道路舗装/奥州市_道路舗装.csv', encoding='shift-jis')